# DATA Preparations for the model

In [25]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import datetime
import math
import glob
from random import *
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [26]:
PATH = '/home/c_thititanapak/latent_image_processed_gae_96/'

high_blur_list_train = [f for f in glob.glob(PATH + 'high_blur_*.BMP')]
high_blur_list_train.sort()

original_list_train = [f for f in glob.glob(PATH + 'original_*.BMP')]
original_list_train.sort()

In [27]:
PATH = '/home/c_thititanapak/latent_image_processed_gae_96_test/'

high_blur_list_test = [f for f in glob.glob(PATH + 'high_blur_*.BMP')]
high_blur_list_test.sort()

original_list_test = [f for f in glob.glob(PATH + 'original_*.BMP')]
original_list_test.sort()

In [28]:
def make_image_list(image_list):
    result_image_list = []

    if (len(image_list) != 0):
        image_list.sort()
        for i in range(len(image_list)):
            temp = cv2.imread(image_list[i], 0)/255.0
#             temp.reshape(96,96,1)
            result_image_list.append(temp)
        
    return result_image_list

In [29]:
train_x = make_image_list(high_blur_list_train)
train_y = make_image_list(original_list_train)

test_x = make_image_list(high_blur_list_test)
test_y = make_image_list(original_list_test)

In [30]:
PATH = '/home/c_thititanapak/latent_image_processed_gae_96/'

In [31]:
train_path_list = [f for f in glob.glob(PATH + "*")]

In [32]:
print(np.shape(train_path_list[0]))

()


# THE gae model

<b><i>input</i></b><br>
<u>Problem latent image of size 96*96</u>

<b><i>output</i></b><br>
<u>Enhanced latent image of size 96*96</u>



In [33]:
# # THE AUTO ENCODER Assume cov2d resize as 0.5 
# def define_autoencoder(image_shape=(96,96,1)):

#     iterations = 0
#     temp = image_shape[0]

#     while(temp %2 == 0):
#         temp = temp/2
#         iterations = iterations+1
#         print(temp)
        
#     print(iterations)
#     input_layer = Input(shape=image_shape)
#     x = input_layer
#     for i in range(iterations):
#         if (i != 0):
#             if (i == iterations-1 and iterations % 2 == 0):
#                 x = Conv2D(N_CONV_FILTER, kernel_size=(3,3), padding='same')(x)
#             else:
#                 x = Conv2D(N_CONV_FILTER/2**(iterations-i), kernel_size=(3,3), strides=(2,2), padding='same')(x)
#         else:
#             x = Conv2D(N_CONV_FILTER/2**(iterations-i), kernel_size=(3,3), strides=(2,2), padding='same')(input_layer)
#         x = BatchNormalization()(x)
#         x = LeakyReLU(alpha=0.2)(x)

#     if (iterations % 2 != 0):
#         x = Conv2D(N_CONV_FILTER, kernel_size=(3,3), padding='same')(x)
#         x = BatchNormalization()(x)
#         x = LeakyReLU(alpha=0.2)(x)
        
#     for i in range(iterations):
#         x = Conv2DTranspose(N_CONV_FILTER/2**i, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
#         x = BatchNormalization()(x)
#         x = ReLU()(x)
        
#     output_layer = Conv2DTranspose(1, kernel_size=(image_shape[0],image_shape[1]), padding='same')(x)
#     model = Model(input_layer,output_layer)
    
#     return model 

In [34]:
# THE AUTO ENCODER Assume cov2d resize as 0.5 
# min_ks = minimum kernel size eg. (3,3)
def define_autoencoder(image_shape=(96,96,1), min_n_filter=32, min_ks=3):
    if image_shape[0] != image_shape[1]:
        print('image shape should be square')
        return
    
    iterations = 0
    temp = image_shape[0]
    
    while(temp % 2 == 0):
        temp = temp / 2
        iterations = iterations + 1
    max_ks = min_ks+(2*iterations)
    # max_ks = 3+(2*5) = 13
    ## ENCODER ##
    input_layer = Input(shape=image_shape)
    x = input_layer
    for i in range(iterations):
        
        asc_n_filter = min_n_filter * (2 ** i)
        desc_kernel_size = (max_ks - (2 * i), max_ks - (2 * i))
        
        if (i != 0):
            x = Conv2D(asc_n_filter, kernel_size=desc_kernel_size, strides=(2,2), padding='same')(x)
        else:
            x = Conv2D(min_n_filter, kernel_size=(max_ks,max_ks), strides=(2,2), padding='same')(input_layer)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)
    
    ## DECODER ##
    for i in range(iterations):
        
        desc_n_filter = min_n_filter * (2 ** (iterations - i - 2))
        asc_kernel_size = (min_ks + (2 * (i + 1)),min_ks + (2 * (i + 1)))
        
        x = Conv2DTranspose(desc_n_filter, kernel_size=asc_kernel_size, strides=(2, 2), padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
   
    
    output_layer = Conv2DTranspose(1, kernel_size=(image_shape[0],image_shape[1]), padding='same')(x)
    model = Model(input_layer,output_layer)
    
    return model 

In [35]:
model = define_autoencoder()

In [36]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 96, 96, 1)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 48, 48, 32)        5440      
_________________________________________________________________
batch_normalization_10 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        247872    
_________________________________________________________________
batch_normalization_11 (Batc (None, 24, 24, 64)        256       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 24, 24, 64)       

In [79]:
def zero_padding(image, padding_size):
    if padding_size <= 0:
        return image

    height = image.shape[0]
    width = image.shape[1]

    zero_padding_image = np.zeros([height + (padding_size * 2), width + (padding_size * 2)], dtype=np.uint8)
    zero_padding_height = zero_padding_image.shape[0]
    zero_padding_width = zero_padding_image.shape[1]

    row_start = padding_size
    col_start = padding_size
    row_end = zero_padding_height - padding_size
    col_end = zero_padding_width - padding_size

    zero_padding_image[row_start:row_end, col_start:col_end] = image

    return zero_padding_image

In [80]:
# def convolute(image, filter, mode='sum'):
#     height, width = image.shape
#     result_image = np.zeros(image.shape, dtype=np.uint8)
#     padding_size = filter.shape[0] // 2
#     zero_padding_image = zero_padding(image, padding_size)
#     plt.imshow(zero_padding_image), plt.show()
#     flipped_filter = np.flip(filter)

#     for i in range(padding_size, height):
#         for j in range(padding_size, width):
#             conv_matrix = zero_padding_image[i-padding_size:i+padding_size+1, j-padding_size:j+padding_size+1] * flipped_filter
#         if mode == 'sum':
#             result_image[i][j] = np.absolute((np.sum(conv_matrix)).astype(np.uint8))
#         elif mode == 'median':
#             result_image[i][j] = np.absolute((np.median(conv_matrix)).astype(np.uint8))
#         elif mode == 'average':
#             result_image[i][j] = np.absolute((np.mean(conv_matrix)).astype(np.uint8))
    
#     return result_image.astype(np.uint8)

In [81]:
def ridge_gradient(images):
#     image = tf.Variable(image)
    
    sobel_0 = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    sobel_45 = np.array([[-1, -1, 0], [-1, 0, 1], [0, 1, 1]])
    sobel_90 = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
    sobel_135 = np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]])
    
    grads_0 = []
    grads_45 = []
    grads_90 = []
    grads_135 = []
    
    for i, image in enumerate(images):
        grad_0 = np.zeros(image.shape, dtype=np.int16)
        grad_45 = np.zeros(image.shape, dtype=np.int16)
        grad_90 = np.zeros(image.shape, dtype=np.int16)
        grad_135 = np.zeros(image.shape, dtype=np.int16)

    #     img = image.numpy()
    #     print(img)
#         print(image.shape)

        grad_0 = cv2.filter2D(image.reshape(96,96), -1, sobel_0)
        grad_45 = cv2.filter2D(image.reshape(96,96), -1, sobel_45)
        grad_90 = cv2.filter2D(image.reshape(96,96), -1, sobel_90)
        grad_135 = cv2.filter2D(image.reshape(96,96), -1, sobel_135)
        
        grads_0.append(grad_0)
        grads_45.append(grad_45)
        grads_90.append(grad_90)
        grads_135.append(grad_135)

    return grads_0, grads_45, grads_90, grads_135
    

In [82]:
#https://stackoverflow.com/questions/3149279/optimal-sigma-for-gaussian-filtering-of-an-image
#http://www.cse.dmu.ac.uk/~sexton/WWWPages/HIPR/html/gsmooth.html

#https://stackoverflow.com/questions/61394826/how-do-i-get-to-show-gaussian-kernel-for-2d-opencv
def gaussian_kernel_smoothing(kernel_len = 21, sigma=3):
    kernel = np.zeros((kernel_len, kernel_len))
    
    # a dirac delta 
    kernel[kernel_len//2, kernel_len//2] = 1
    
    # a gaussian blur of dirac delta is the 2D gaussian mask filter
    return cv2.GaussianBlur(kernel, (kernel_len,kernel_len), sigmaX=sigma, sigmaY=sigma)
    

In [86]:
def ridge_orientation(images):
#     image = tf.Variable(image)
    gauss_block = np.array([gaussian_kernel_smoothing(7, 1)], dtype= np.float32)
    gauss_orient = np.array([gaussian_kernel_smoothing(31, 5)], dtype= np.float32)
    
    sobel_0 = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    sobel_90 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    
    # Get gaussian gradient
#     gauss_gradient_x = cv2.filter2D(gauss_block, -1, sobel_0)
#     gauss_gradient_y = cv2.filter2D(gauss_block, -1, sobel_90) # >>> 0 ???
#     gauss_gradient_y = cv2.filter2D(np.transpose(gauss_block), -1, sobel_0)
    gauss_gradient_x = cv2.filter2D(gauss_block.reshape(7,7), -1, sobel_0)
    gauss_gradient_y = cv2.filter2D(gauss_block.reshape(7,7), -1, sobel_90)
    
#     print("G Y")
#     print(gauss_gradient_y, gauss_gradient_x)
    
    # Get image gradients
#     print(image.shape)
    
    orient_imgs = []
    reliabilities = []
    for i, image in enumerate(images):
        
        gauss_img_x = cv2.filter2D(image.reshape(96,96), -1, gauss_gradient_x.reshape(7,7))
        gauss_img_y = cv2.filter2D(image.reshape(96,96), -1, gauss_gradient_y.reshape(7,7))

    #     plt.imshow(gauss_img_x,'gray'), plt.show()
    #     plt.imshow(gauss_img_y,'gray'),plt.show()

        gauss_img_xx = gauss_img_x ** 2
        gauss_img_xy = gauss_img_x * gauss_img_y
        gauss_img_yy = gauss_img_y ** 2

    #     print(gauss_img_xy[45])
    #     plt.imshow(gauss_img_xy,'gray'), plt.show()

        gauss_img_xx = cv2.filter2D(gauss_img_xx, -1, gauss_orient.reshape(31,31))
        gauss_img_xy = 2 * cv2.filter2D(gauss_img_xy, -1, gauss_orient.reshape(31,31))
        gauss_img_yy = cv2.filter2D(gauss_img_yy, -1, gauss_orient.reshape(31,31))
    #     print(gauss_img_xy)
    #     plt.imshow(gauss_img_xy,'gray'), plt.show()

        # get orient img
        denom = np.sqrt(gauss_img_xy**2 + (gauss_img_xx - gauss_img_yy)**2 + 1**(-12))

        sin2theta = gauss_img_xy / denom
        cos2theta = (gauss_img_xx - gauss_img_yy) / denom

        # smooth double angle of orient img
        sin2theta = cv2.filter2D(sin2theta, -1, gauss_orient.reshape(31,31))
        cos2theta = cv2.filter2D(cos2theta, -1, gauss_orient.reshape(31,31))
    #     print(sin2theta, cos2theta)

    #     print('log')
    #     print(np.pi/2.0 + np.arctan2(sin2theta, cos2theta) / 2.0)

        orient_img = np.pi/2.0 + np.arctan2(sin2theta, cos2theta) / 2.0
    #     plt.imshow(orient_img), plt.show()
        # Imin Imax Rscore and coherence
        Imin = (gauss_img_yy + gauss_img_xx) / 2.0 - (gauss_img_xx - gauss_img_yy) * cos2theta / 2.0 - gauss_img_xy * sin2theta / 2.0 
        Imax = gauss_img_yy + gauss_img_xx - Imin

        reliability = 1 - Imin / (Imax + 0.001)
    #     print('denom HERE!')
    #     print(denom.shape)
        if (denom.all() <= 0.001):
            reliability = 0

        coherence = ((Imax - Imin) / (Imax + Imin + 1 ** (-12))) ** 2
        
        orient_imgs.append(orient_img)
        reliabilities.append(reliability)
    
    return orient_imgs, reliabilities, coherence
    
    

In [87]:
def mse(a,b):
    return tf.reduce_mean((a -b)**2)

In [88]:
#https://towardsdatascience.com/custom-loss-function-in-tensorflow-2-0-d8fa35405e4e
#https://stackoverflow.com/questions/34875944/how-to-write-a-custom-loss-function-in-tensorflow
def loss_fn(input_img_tf, output_img_tf):
    # -- get numpy of tf -- #
    input_img = input_img_tf.numpy()
    output_img = output_img_tf.numpy()
    
    #---- gradient loss ----#
    grad_0_in, grad_45_in, grad_90_in, grad_135_in = tf.convert_to_tensor(ridge_gradient(input_img))
    grad_0_out, grad_45_out, grad_90_out, grad_135_out = tf.convert_to_tensor(ridge_gradient(output_img))
    

    cost0 = tf.keras.losses.MSE(grad_0_out, grad_0_in)
    cost45 = tf.keras.losses.MSE(grad_45_out, grad_45_in)
    cost90 = tf.keras.losses.MSE(grad_90_out, grad_90_in)
    cost135 = tf.keras.losses.MSE(grad_135_out, grad_135_in)
#     cost0 = mse(grad_0_out, grad_0_in)
#     cost45 = mse(grad_45_out, grad_45_in)
#     cost90 = mse(grad_90_out, grad_90_in)
#     cost135 = mse(grad_135_out, grad_135_in)
    
    grad_loss_sum = tf.math.add_n([cost0, cost45, cost90, cost135])
    grad_loss = tf.math.divide(grad_loss_sum, 4)

    # TODO OVER HERE >>> แก้ ride_orient
    
    #---- orientation loss and reliability loss ----#
    ori_in, rel_in, _ = ridge_orientation(input_img)
    ori_out, rel_out, _ = ridge_orientation(output_img)
    
    ori_in_tf = tf.convert_to_tensor(ridge_orientation(input_img))
    rel_in_tf = tf.convert_to_tensor(ridge_orientation(input_img))
    ori_out_tf = tf.convert_to_tensor(ridge_orientation(input_img))
    rel_out_tf = tf.convert_to_tensor(ridge_orientation(output_img))
    
    ori_loss = tf.keras.losses.MSE(ori_out, ori_in)
    rel_loss = tf.keras.losses.MSE(rel_out, rel_in)
#     ori_loss = mse(ori_out, ori_in)
#     rel_loss = mse(rel_out, rel_in)
      
    sum_loss = tf.math.add_n([grad_loss, ori_loss, rel_loss])
    loss = tf.math.divide(sum_loss, 3)
    
    return loss
#, grad_loss, ori_loss, rel_loss

In [89]:
tf_loss_fn(train_x[0], train_x[0])

NameError: name 'tf_loss_fn' is not defined

# Training the model

In [90]:
#Hyper Parameters for Adam
weight_decay = 0        
momentum = 0.5          
num_ch = 1     # num of channels in image
lr = 0.0002       # initial learning rate

In [91]:
def train(model, train_x, train_y, batch_size, epochs, exp_id):
#     img = tf.Variable(img)
# opt = tf.optimizers.Adam(learning_rate=lr, decay = 1e-6)

# for _ in range(epoch):
#     with tf.GradientTape() as tape:
#         tape.watch(img)
#         y = model(img.value())[:, :, :, filter]
#         loss = -tf.math.reduce_mean(y)

#     grads = tape.gradient(loss, img)
#     opt.apply_gradients(zip([grads], [img]))
    optimizers = Adam(learning_rate=lr, beta_1 = momentum, decay = weight_decay)

    batch_round = math.ceil(len(train_x) / batch_size)
    
    loss = []
    grad_loss = []
    ori_loss = []
    rel_loss = []
    
    for epoch in range(epochs):
        print('epoch:', epoch, 'start...')
        for batch in range(batch_round):
            start_time = datetime.datetime.now()
            
            batch_start = batch * batch_size
            batch_end = (batch_start + batch_size) - 1
            
            image_x_batched = tf.convert_to_tensor(train_x[batch_start: batch_end])
            image_y_batched = tf.convert_to_tensor(train_y[batch_start: batch_end])
            
            for i in range(1):
                print(i)
                with tf.GradientTape() as tape:
#                     print('HERE')
                    tape.watch(model.trainable_variables)
                    enhanced_image = model(tf.reshape(image_x_batched, (-1,96,96)))
#                     print(enhanced_image.shape)
#                     print(image_y_batched.shape)
                
                    total_loss = loss_fn(tf.reshape(image_y_batched, (image_y_batched.shape[0],image_y_batched.shape[1],image_y_batched.shape[2],1)), enhanced_image)

#                     total_loss = tf.reduce_mean(enhanced_image)
#                     print(total_loss)
                  
                gradients_of_model = tape.gradient(total_loss, model.trainable_variables)
                
                print('grad of model')
                print(gradients_of_model)

                optimizers.apply_gradients(zip(gradients_of_model, model.trainable_variables))
                return 0, 0, 0, 0
                
            loss.append(total_loss)
#             grad_loss.append(grad_l)
#             ori_loss.append(ori_l)
#             rel_loss.append(rel_l)
            elapsed_time = datetime.datetime.now() - start_time
            print('loss:', total_loss)
            print('batch:', batch, 'elapsed time:', elapsed_time)
        
    # SAVE MODELS
    model.save(WEIGHT_SAVING_PATH + 'gae_'+ str(exp_id) +'.h5')
            
    return loss, grad_loss, ori_loss, rel_loss

In [92]:
loss, _,_,_ = train(model, train_x, train_y, 22, 1, 1)

epoch: 0 start...
0


ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [20]:
def tf_loss_fn(input_img, output_img):

#     print('tf loss fn')
#     print(type(tf.convert_to_tensor(input_img)), input_img.shape)
#     print(type(output_img), output_img.shape)
    
    return tf.reduce_mean(input_img)
    
    

In [ ]:
loss